This notebook is for making predictions on unlabeled data. You can annotate pictures with the predicted center using this notebook.

In [104]:
import PIL
print('Pillow Version:', PIL.__version__)

Pillow Version: 5.4.1


In [105]:
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import keras.preprocessing.image as preprocess
import os

In [215]:
# parameters
#path = "/home/hippoc/bin/beauvoir/data/thin_distract_pics/far/"

path = '/home/hippoc/schen/testing_vids/test5/resized/' # The path where images to annotate are located
image_num = 459 # The number of images to annotate
save_folder = "preds_v5/" # The folder to save annotated images
model_path = '/home/hippoc/schen/models/v5/weights.04-14.36.hdf5' # The location of the model weights


#image_num = 8000
#images = np.zeros((image_num, 200, 300, 3))
#for i in range(image_num):
#    img = preprocess.load_img(path + str(i)+'.png')
#    img_array = preprocess.img_to_array(img)
#    images[i] = img_array
    
# preprocessing images (range from 0 to 1)
#images /= 255.0

#datagen = ImageDataGenerator(
#        featurewise_center=True,
#        samplewise_center=False,
#        featurewise_std_normalization=True,
#        samplewise_std_normalization=False,
#        zca_whitening=True,
#        rotation_range=0,
#        width_shift_range=0,
#        height_shift_range=0,
#        horizontal_flip=False,
#        vertical_flip=False
#        )



In [89]:
input_shape = (200, 300, 3) # (height, width, channels) of image input

The model is defined in the next two blocks:

In [100]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Input, BatchNormalization, Activation
import keras.layers
from keras.losses import mean_squared_error
from keras.optimizers import Adam
from keras.callbacks import History

def ResLayer(x, filters):
    # identity
    res = x
    res = BatchNormalization()(res)
    res = Conv2D(filters=filters, kernel_size=[1,1], strides=2, padding='same', use_bias=False)(res)
    
    # conv layers
    out = BatchNormalization()(x)
    out = Activation('relu')(out)
#    out = Activation('relu')(x)
    out = Conv2D(filters=filters, kernel_size=[3,3], strides=2, padding='same', use_bias=False)(out)
    
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(filters=filters, kernel_size=[3,3], strides=1, padding='same', use_bias=False)(out)
    
    # adding the identity
    out = keras.layers.add([res,out])
    
    return out

In [101]:
# input layer
inputs = Input(shape=input_shape)

x = Conv2D(filters=32, kernel_size=[5,5], strides=2, padding='same')(inputs)
x = MaxPooling2D(pool_size=(3,3), strides=2)(x)

x = ResLayer(x, 32)
x = ResLayer(x, 64)
x = ResLayer(x, 128)

#x = Dropout(0.5)(x)
x = Activation('relu')(x)
x = Flatten()(x)

x = Dense(2)(x)

model = Model(inputs=inputs, outputs=x)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 100, 150, 32) 2432        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 49, 74, 32)   0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 49, 74, 32)   128         max_pooling2d_2[0][0]            
__________________________________________________________________________________________________
activation

In [8]:
import keras.backend as K

def mean_mse(y_true, y_pred):
    return K.mean(K.sum(K.square(y_true-y_pred), axis=-1))

# Custom loss metric (returns the average distance off per image)

In [216]:
# compile model
model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.0015),
              metrics=[mean_mse]
              )

# load model weights
model.load_weights(model_path)

In [220]:
# making annotations on training data

images = np.zeros((image_num, input_shape[0], input_shape[1], input_shape[2]))


for i in range(1,image_num+1):
    img = preprocess.load_img(path + str(i) + ".png")
    img_array = preprocess.img_to_array(img)
    images[i-1] = img_array
    
# preprocessing images (range from 0 to 1)
images /= 255.0


x_pred = images
preds = model.predict(x_pred, verbose=1) # predictions range from -1 to 1
# bottom left corner is (-1, -1), top right is (1, 1)

KeyboardInterrupt: 

In [218]:
preds[:,0] = preds[:,0]*150 + 150 # converting x-coordinate to range from 0-300
preds[:,1] = preds[:,1]*-100 + 100 # converting y-coordinate to range from 0-200
preds = np.round(preds)

In [219]:
# making annotations on images

from PIL import ImageDraw

for i in range(1,image_num+1):
    img = Image.open(path + str(i) + ".png")
    draw = ImageDraw.Draw(img)
    
    pred = preds[i-1]
    
    draw.line([tuple(pred - [10, 0]), tuple(pred + [10, 0])], fill="blue", width=3)
    draw.line([tuple(pred - [0, 10]), tuple(pred + [0, 10])], fill="red", width=3)

#    draw.point(labels_orig[i], fill="blue")
#    draw.point(preds[i-15840], fill="black")
    img.save(path + save_folder + str(i) + "_pred.png")

In [125]:
#model.save('/home/hippoc/schen/models/full_models/300_200_full_v4_e2.h5')